
# The Notebook will perform Unity Catalog Tool Creation

* UC function tools are managed in Unity Catalog with build-in security and compliance features
* Refer to [AI Agent Tools](https://docs.databricks.com/aws/en/generative-ai/agent-framework/agent-tool)

In [0]:
%pip install databricks-sdk -U -q
dbutils.library.restartPython()

In [0]:
%run ./config

In [0]:
dbutils.widgets.text(name="recreate_uc_tools", label="Recreate UC Tools", defaultValue="false")

In [0]:
recreate_uc_tools = dbutils.widgets.get("recreate_uc_tools")
if recreate_uc_tools == "true":
    recreate_uc_tools = True
else:
    recreate_uc_tools = False
print(f"recreate_uc_tools: {recreate_uc_tools}")


# Check the existing tools in the target schema

In [0]:
def show_or_clean_uc_tools(catalog, schema, delete_functions=False):
    df = spark.sql(f"SHOW USER FUNCTIONS IN {catalog}.{schema};")
    count = df.count()
    rows = df.collect()
    functions = [r.function for r in rows]
    print(f"There are {count} user defined UC functions: {functions}")
    if count > 0 and delete_functions:
        for function_name in functions:
            spark.sql(f"DROP FUNCTION if exists {function_name};")
            print(f"Dropped {function_name}")
        print(f'{spark.sql(f"SHOW USER FUNCTIONS IN {catalog}.{schema};").count()} functions exist')

In [0]:
spark.sql(f"USE CATALOG {catalog}");
spark.sql(f"USE SCHEMA {schema}");
show_or_clean_uc_tools(catalog, schema, delete_functions=recreate_uc_tools)

# Create UC Functions

## Get Customer Policy Profile from Phone Number

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_customer_policy_profile_by_phone_number(
  search_name STRING
)
RETURNS TABLE (
  profile STRING
)
COMMENT 'Return a customer policy profile with a given phone number'
LANGUAGE SQL
  RETURN
  SELECT
    CONCAT("The customer ", first_name, " ", last_name, " lives in ", address, " with email address ",
           email, " and phone number", phone_number, " owns a ", make, " ", model_year, " ", model, " has a policy of ", 
           policy_type, " (policy number:", policy_number, ") issued on ", issue_date, " and expires on ", expiration_date)
  as policy_profile
  FROM call_center_transcripts_analysis
  where phone_number = search_name
  limit 1
  ;

select * FROM get_customer_policy_profile_by_phone_number('(227)-636-7543') 

## Get Customer Intent by Phone Number

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_customer_intent_by_phone_number(
  search_name STRING
)
RETURNS TABLE (
  profile STRING
)
COMMENT 'Return the customer intent of the last call transcript with a given phone number'
LANGUAGE SQL
  RETURN
  select
    intent
  from call_center_transcripts_analysis
  where phone_number = search_name
  order by call_timestamp desc
  limit 1
;

select * from get_customer_intent_by_phone_number('(875)-209-8555')

## Get Customer Sentiment by Phone Number

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_customer_sentiment_by_phone_number(
  search_name STRING
)
RETURNS TABLE (
  profile STRING
)
COMMENT 'Return the customer sentiment of the last call transcript with a given phone number'
LANGUAGE SQL
  RETURN
  select
    sentiment
  from call_center_transcripts_analysis
  where phone_number = search_name
  order by call_timestamp desc
  limit 1
;

select * from get_customer_sentiment_by_phone_number('(875)-209-8555')

## Get Customer call transcript by phone number

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_customer_transcript_by_phone_number(
  search_name STRING
)
RETURNS TABLE (
  profile STRING
)
COMMENT 'Return the customer last call transcript with a given phone number'
LANGUAGE SQL
  RETURN
  select
    transcript
  from call_center_transcripts_analysis
  where phone_number = search_name
  order by call_timestamp desc
  limit 1
;

select * from get_customer_transcript_by_phone_number('(875)-209-8555')